In [50]:
import numpy as np
import pandas as pd
import re
from glob import glob
from string import punctuation
from time import time
from sys import intern
from collections import Counter

# https://medium.com/@asheeshmisra29/web-automation-selenium-webdriver-and-python-getting-started-part-3-a9c07143d36d
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import scrapy
from scrapy.crawler import CrawlerProcess

from string_grouper import match_strings, match_most_similar, group_similar_strings, StringGrouper
# from sklearn.feature_extraction.text import TfidfVectorizer
# from scipy.sparse import csr_matrix
# import sparse_dot_topn.sparse_dot_topn as sdt

cosmetic ingredient database: https://public.opendatasoft.com/explore/dataset/cosmetic-ingredient-database-ingredients-and-fragrance-inventory/table/

cosdna: https://cosdna.com/

In [2]:
file_list = glob('./data/*.csv')
file_list

['./data\\brands.csv',
 './data\\recommendations_2020-05-05.csv',
 './data\\responses_2020-05-05.csv',
 './data\\responses_modified_2020-05-05.csv']

In [3]:
responses = pd.read_csv(file_list[2])

In [4]:
new_columns = [
    'timestamp',
    'email_address',
    'name',
    'order_number',
    'skin_type',
    'cannot_contain',
    'skin_concern',
    'am_routine',
    'pm_routine',
    'climate',
    'skin_sensitivities',
    'used_retinoids',
    'used_acids',
    'prone_to_breakouts',
    'miscellaneous',
    'permission'
]

responses.columns = new_columns
responses.drop(columns=['email_address','name'], inplace=True)
responses.head()

,timestamp,order_number,skin_type,cannot_contain,skin_concern,am_routine,pm_routine,climate,skin_sensitivities,used_retinoids,used_acids,prone_to_breakouts,miscellaneous,permission
0,4/30/2020 12:43:32,12846,Combination,None,"Acne, Pigmentation, Fine lines/wrinkles","Trader Joe’s “all in one” cleanser, Bioderma S...","Trader Joe’s “all in one” cleanser, Bioderma S...","Temperate, a lovely 70F (lucky!)",I broke out from Cosrx snail serum and from Ma...,Differing 0.1% for four years,I prefer toners with acids as masks with AHA s...,I have mild to moderate acne that I have been ...,"I have a super oily forehead, oily nose and c...",Yes
1,4/30/2020 12:45:25,12837,Combination,No allergies,"Acne, Fine lines/wrinkles, Skin Texture (rough...",1. Wash with Biore Charcoal Acne Clearing Clea...,Collapse into bed,"Tropical, humid",No.,No.,No.,Yes. They are usually cystic and around my cyc...,I need help.,Yes
2,4/30/2020 12:50:26,12845,Oily,No.,"Acne, Pigmentation",Cleanser (Drunk Elephant Beste No. 9 Jelly Cle...,Oil cleanser (Dermalogica Precleanse); Cleanse...,Desert dry heat,No.,No.,I've used 2% BHA on and off; and daily I use a...,"Yes, occasional small whiteheads and blackhead...",I'm Black so hyperpigmentation and skin discol...,Yes
3,4/30/2020 13:02:43,12850,Normal,No,"Fine lines/wrinkles, Skin Texture (roughness, ...","Splash of water, occasional Paula's Choice enr...",Occasional Palmer's facial cleansing oil (if w...,"Tropical, humid",No,Almost through the purging/peeling phase with ...,I've tried (and still own) The Ordinary red pe...,"Not really - a very rare pimple, I tend more o...",I turn 43 in a month and am really trying to g...,Yes
4,4/30/2020 13:08:20,12842,Combination,"Ethylhexyl palmitate, strong sulfates, chemica...","Acne, Skin Texture (roughness, dullness), Other","Osea Ocean Cleanser, Renee Rouleau (RR) Elderb...","Renee Rouleau (RR) Luxe Mint Cleanser, Renee R...","Temperate, a lovely 70F (lucky!)",Overall I have fairly sensitive skin with horm...,"No, but I would like to start to address early...",Azelaic acid has been a good acid for me (not ...,"My acne is adult onset, primarily hormonal, an...","I just moved to Chicago, so lots of temperatur...",Yes


In [5]:
responses['skin_concern'] = responses['skin_concern'].str.lower().str.split(', ')
responses['skin_concern'][:5]

0            [acne, pigmentation, fine lines/wrinkles]
1    [acne, fine lines/wrinkles, skin texture (roug...
2                                 [acne, pigmentation]
3    [fine lines/wrinkles, skin texture (roughness,...
4    [acne, skin texture (roughness, dullness), other]
Name: skin_concern, dtype: object

## using requests-html

https://www.jcchouinard.com/web-scraping-with-python-and-requests-html/

In [65]:
import requests
from requests_html import HTMLSession

url = 'https://cosdna.com/eng/product.php?q=la+roche+posay+antihelios+mineral'

try:
    session = HTMLSession()
    response = session.get(url)
    
except requests.exceptions.RequestException as e:
    print(e)

2020-05-18 13:34:19 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): cosdna.com:443
2020-05-18 13:34:21 [urllib3.connectionpool] DEBUG: https://cosdna.com:443 "GET /eng/product.php?q=la+roche+posay+antihelios+mineral HTTP/1.1" 200 None


In [72]:
result = response.html.find('.pl-0')
print(result[0].text)

LA ROCHE-POSAY Antihelios mineral sunscreen spf 50


# cosdna object

https://selenium-python.readthedocs.io/page-objects.html
https://www.youtube.com/watch?v=BURK7wMcCwU

In [6]:
ALIASES = {}
BASE_NAMES = set()

In [19]:
class CosDNA():
    
    session_open = False
    CHROMEDRIVER_PATH = 'C:/chromedriver_win32/chromedriver.exe'
    MAIN_URL = 'cosdna.com'
    
    sort_dict = {
        'latest': '&sort=date',
        'featured': '&sort=featured',
        'clicks': '&sort=click',
        'reviews': '&sort=review'
    }

    def __init__(self):
        if not CosDNA.session_open:
            self.start()
    
    @classmethod
    def start(cls):
        cls.driver = webdriver.Chrome(CosDNA.CHROMEDRIVER_PATH)
    
    @classmethod
    def end(cls):
        cls.driver.quit()

In [20]:
class Cosmetic(CosDNA):
    
    # include cosdna_url parameter to make things easier for ingredients later
    def __init__(self, name=None, cosdna_url=None):
        # opens webdriver.Chrome
        super().__init__()          
        # synced = True only after running self.sync()
        self.synced = False         
        self.__name = name
        self.__cosdna_url = cosdna_url
            
    def link(self, base_url=None, sort=None, cosdna_url=None):
        # if we want to use cosdna_url at this step
        if cosdna_url is not None:
            self.__cosdna_url = cosdna_url
            # skip searching since we already have the url
            return self
        else:
            # search for self.name
            # base_url defined in child classes
            search_name = self.__name.lower()
            search_name = search_name.translate(str.maketrans('', '', punctuation))
            search_name = search_name.replace(' ', '+')
            # products have different sorts
            if sort in CosDNA.sort_dict.keys():
                search_url = base_url + search_name + CosDNA.sort_dict[sort]
            # ingredients do not
            else:
                search_url = base_url + search_name
            CosDNA.driver.get(search_url)
            # try to get the top result from the search
            if 'cosmetic' in CosDNA.driver.title.lower():
                try:
                    top = CosDNA.driver.find_element_by_xpath(("//table[@class='table table-hover']/tbody/tr/td[1]/a"))
                    self.__cosdna_url = top.get_attribute('href')
                    return self
                except:
                    print(f'No results for {self.name} on CosDNA.')
                    name = input('Enter new search: ')
                    if name.lower() == 'break':
                        return self
                    else:
                        self.name = name
                        return self.link(base_url=base_url, sort=sort)
            else:
                self.__cosdna_url = CosDNA.driver.current_url
                return self
        
    def sync(self):
        if self.cosdna_url is None:
            print('Cannot sync without first linking with valid CosDNA URL.')
            return self        
        else:
            CosDNA.driver.get(self.cosdna_url)
            return self
    
    @property
    def cosdna_url(self):
        if self.__cosdna_url is None:
            return self.__cosdna_url
        elif CosDNA.MAIN_URL in self.__cosdna_url:
            return self.__cosdna_url
        else:
            print('Invalid CosDNA URL')
            return None
    
    @property
    def linked(self):
        return (self.cosdna_url is not None)

In [21]:
test = Cosmetic()

In [22]:
test.session_open

False

In [24]:
test.end()

In [23]:
test2 = Cosmetic()

In [26]:
test2.session_open

False

In [25]:
test2.end()

In [58]:
#         self.__repr_name, self.__repr_brand, self.__repr_product, self.__repr_cosdna_url = \
#         name, brand, product, cosdna_url

#     def __repr__(self):
#         return f"Product(name={self.__repr_name}, brand={self.__repr_brand}, product={self.__repr_product}, cosdna_url={self.__repr_cosdna_url})"

In [12]:
class Ingredient(Cosmetic):
    
    def __init__(self, name=None, cas_id=None, cosdna_url=None, function=None):
        self.__name = name
        self.cas_id = cas_id
        self.function=function
        super().__init__(self.__name, cosdna_url=cosdna_url)
        
    def link(self, cosdna_url=None):
        if cosdna_url is None:
            cosdna_url = self.cosdna_url
        else:
            cosdna_url = cosdna_url
        super().link(base_url='https://cosdna.com/eng/stuff.php?q=',
                     cosdna_url=cosdna_url)
        
    def sync(self):
        super().sync()
        self.cosdna_name = CosDNA.driver.find_element_by_class_name('text-vampire').text.lower()
        if self.cosdna_name not in BASE_NAMES:
            BASE_NAMES.add(self.cosdna_name)
        self.aliases = CosDNA.driver.find_element_by_class_name('mb-2').text.lower().split(', ')
        for alias in self.aliases:
            if alias not in [*ALIASES]:
                ALIASES[alias] = self.cosdna_name
        info = CosDNA.driver.find_element_by_class_name('justify-content-between')
        try:
            mw = info.find_elements_by_xpath("//*[contains(text(), 'Molecular Weight')]")[0].text
            self.mass = float(re.findall("Molecular Weight: (\d*.\d*)", mw)[0])
        except:
            self.mass = None
        try:
            cas = info.find_elements_by_xpath("//*[contains(text(), 'Cas')]")[0].text
            self.cas_id = re.findall("Cas No：(.*)", cas)[0]
        except:
            self.cas_id = None
        # self.synced = False inherited from Cosmetic.__init__()
        self.synced = True
        return self
        
    def link_sync(self, cosdna_url=None):
        self.link(cosdna_url=cosdna_url)
        self.sync()
        return self

        
    @property
    def name(self):
        if self.synced:
            return self.cosdna_name
        else:
            return self.__name

In [13]:
class ProductDataFrame(Cosmetic):
    
    def __init__(self, name=None, brand=None, product=None, cosdna_url=None):

        self.brand = brand
        self.product = product
        # __name vs. name property of concatenating brand + product
        self.__name = name
        # call super().__init__ using name property
        super().__init__(self.name, cosdna_url=cosdna_url)
    
    def link(self, sort='featured', cosdna_url=None):
        if cosdna_url is None:
            cosdna_url = self.cosdna_url
        else:
            cosdna_url = cosdna_url
        return super().link(base_url='https://cosdna.com/eng/product.php?q=', 
                            sort=sort,
                            cosdna_url=cosdna_url)
        
    def sync(self):
        super().sync()
        cosdna_brand = CosDNA.driver.find_element_by_class_name('brand-name').text.lower()
        cosdna_product = CosDNA.driver.find_element_by_class_name('prod-name').text.lower()
        if cosdna_brand is not '':
            self.brand = cosdna_brand
            self.product = cosdna_product
        cosdna_name = cosdna_brand + ' ' + cosdna_product
        self.__name = cosdna_name.strip()
        # get ingredients information from ingredients table
        self.__ingredients = []
        self.__objects = []
        table = CosDNA.driver.find_element_by_class_name('chem-list')
        rows = table.find_elements_by_tag_name('tr')
        for row in rows:
            ingredient_cell = row.find_elements_by_tag_name('td')[0]
            if 'no results' in ingredient_cell.text.strip().lower():
                ingredient = row.find_elements_by_class_name('text-muted')[0].text.strip().lower()
                function = None
            else:
                ingredient = ingredient_cell.text.strip().lower()
                ingredient_url = ingredient_cell.find_elements_by_tag_name('a')[0].get_attribute('href')
                if len(row.find_elements_by_tag_name('td')) > 1:
                    function_cell = row.find_elements_by_tag_name('td')[1]
                    function = function_cell.text.strip().lower().split(',')
                    if 'sunscreen' in function:
                        try:
                            uva = re.search("uv[ab]\d", 
                                            function_cell.find_elements_by_tag_name('img')[0]\
                                            .get_attribute('src'))[0]
                            uvb = re.search("uv[ab]\d", 
                                            function_cell.find_elements_by_tag_name('img')[1]\
                                            .get_attribute('src'))[0]
                            function.append(uva)
                            function.append(uvb)
                        except:
                            continue
                else:
                    function = None
            self.__ingredients.append(ingredient)
            self.__objects.append(Ingredient(name=ingredient,
                                             cosdna_url=ingredient_url,
                                             function=function))
            self.ingredients = {'ingredient': self.__ingredients, 
                                'objects': self.__objects}
            self.ingredients = pd.DataFrame.from_dict(self.ingredients)
        # self.synced = False inherited from Cosmetic.__init__()
        self.synced = True
        return self
    
    def link_sync(self, sort='featured', cosdna_url=None):
        self.link(sort=sort, cosdna_url=cosdna_url)
        self.sync()
    
    @property
    def name(self):
        if self.brand is None and self.product is None:
            return self.__name
        else:
            self.__name = self.brand + ' ' + self.product
            return self.__name
        
    def __str__(self):
        return self.name


In [14]:
class Product(Cosmetic):
    
    def __init__(self, name=None, brand=None, product=None, cosdna_url=None):

        self.brand = brand
        self.product = product
        # __name vs. name property of concatenating brand + product
        self.__name = name
        # call super().__init__ using name property
        super().__init__(self.name, cosdna_url=cosdna_url)
    
    def link(self, sort='featured', cosdna_url=None):
        if cosdna_url is None:
            cosdna_url = self.cosdna_url
        else:
            cosdna_url = cosdna_url
        return super().link(base_url='https://cosdna.com/eng/product.php?q=', 
                            sort=sort,
                            cosdna_url=cosdna_url)
        
    def sync(self, deep=True):
        super().sync()
        cosdna_brand = CosDNA.driver.find_element_by_class_name('brand-name').text.lower()
        cosdna_product = CosDNA.driver.find_element_by_class_name('prod-name').text.lower()
        if cosdna_brand is not '':
            self.brand = cosdna_brand
            self.product = cosdna_product
        cosdna_name = cosdna_brand + ' ' + cosdna_product
        self.__name = cosdna_name.strip()
        # get ingredients information from ingredients table
        self.ingredients = []
        self.__functions = []
        self.__ingredient_urls = []
        table = CosDNA.driver.find_element_by_class_name('chem-list')
        rows = table.find_elements_by_tag_name('tr')
        for row in rows:
            ingredient_cell = row.find_elements_by_tag_name('td')[0]
            if 'no results' in ingredient_cell.text.strip().lower():
                ingredient = row.find_elements_by_class_name('text-muted')[0].text.strip().lower()
                function = None
            else:
                ingredient_url = ingredient_cell.find_elements_by_tag_name('a')[0].get_attribute('href')
                if len(row.find_elements_by_tag_name('td')) > 1:
                    function_cell = row.find_elements_by_tag_name('td')[1]
                    function = function_cell.text.strip().lower().split(',')
                    if 'sunscreen' in function:
                        try:
                            uva = re.search("uv[ab]\d", 
                                            function_cell.find_elements_by_tag_name('img')[0]\
                                            .get_attribute('src'))[0]
                            uvb = re.search("uv[ab]\d", 
                                            function_cell.find_elements_by_tag_name('img')[1]\
                                            .get_attribute('src'))[0]
                            function.append(uva)
                            function.append(uvb)
                        except:
                            continue
                else:
                    function = None
            ingredient = ingredient_cell.text.strip().lower()
            self.ingredients.append(ingredient)
            self.__functions.append(function)
            self.__ingredient_urls.append(ingredient_url)
#             self.ingredients[ingredient] = Ingredient(name=ingredient, 
#                                                       cosdna_url=ingredient_url,
#                                                       function=function)
#         if deep:
#             self.__objects = []
#             for i, ingredient in enumerate(self.ingredients):
#                 if ingredient not in :
#                     alias = Ingredient(cosdna_url=self.__ingredient_urls[i]).sync().name
#                     ALIASES[ingredient] = 
#             self.ingredients = [ALIASES[ingredient] for ingredient in self.ingredients]
        # self.synced = False inherited from Cosmetic.__init__()
        self.synced = True
        return self
    
    def link_sync(self, sort='featured', cosdna_url=None, deep=True):
        self.link(sort=sort, cosdna_url=cosdna_url)
        self.sync(deep=deep)
    
    @property
    def name(self):
        if self.brand is None and self.product is None:
            return self.__name
        else:
            self.__name = self.brand + ' ' + self.product
            return self.__name
        
    def __str__(self):
        return self.name


In [15]:
def product_test(product):
    
    tick = time()
    print(product.linked)
    print(product.cosdna_url)
    product.link_sync(deep=True)
    print()
    print(product.linked)
    print(product.name)
    print(product.cosdna_url)
#     print(product.ingredients)
    tock = time()
    print()
    print(tock - tick)
    
    return product.ingredients

In [17]:
product = Product(brand='chemist confessions', product='better oil')

In [18]:
product.session_open

False

In [ ]:
product.link_sync(deep=True)
product.ingredients

In [34]:
ALIASES

{'hippophae rhamnoides seed oil': 'hippophae rhamnoides seed oil',
 'ribes nigrum seed oil': 'ribes nigrum seed oil',
 'rosa rubiginosa seed oil': 'rosa rubiginosa seed oil',
 'squalane': 'squalane',
 'bisabolol': 'alpha bisabolol',
 'linoleic acid': 'linoleic acid',
 'linolenic acid': 'linolenic acid',
 'α-tocopherol': 'tocopherols',
 'pelargonium graveolens oil': 'pelargonium graveolens oil',
 'salvia sclarea oil': 'salvia sclarea oil',
 'chamaecyparis obtusa oil': 'chamaecyparis obtusa oil'}

In [ ]:
product_1_dict = ProductDict(brand='chemist confessions', product='better oil')
product_test(product_1_dict)

In [28]:
product2 = Product('chemist confessions aquafix')

product_test(product2)

False
None

True
chemist confessions aquafix
https://cosdna.com/eng/cosmetic_d03f406994.html

6.014865398406982


['water',
 'propanediol',
 'glycerol',
 'ethoxydiglycol',
 'sodium pca',
 'dexpanthenol',
 'allantoin',
 'vinyl dimethicone/methicone silsesquioxane crosspolymer',
 'madecassoside',
 'asiaticoside',
 'polyacrylate crosspolymer-6',
 'sodium hyaluronate',
 'xantham gum',
 'lecithine',
 'sclerotium gum',
 'pullulan',
 'hexanediol',
 'caprylhydroxamic acid',
 'edetate disodium']

In [ ]:
product3 = Product(cosdna_url='https://cosdna.com/eng/cosmetic_57b4385905.html')

product_test(product3)

In [ ]:
class Routine(CosDNA):
    
    def __init__(self, name=None, routine=None):
        super().__init__()
        self.name = name
        if routine is None:
            self.routine = []
        else:
            self.routine = []
            self.add(routine)
        
    def add(self, routine):
        routine = np.array([routine])
        routine = routine.ravel()
        for prod in routine:
            if type(prod) == Product:
                self.routine.append(prod)
            else:
                self.routine.append(Product(prod))
        
        return self
    
    def link(self, sort='featured', force=False):
        
        for product in self.routine:
            if force:
                product.link(sort=sort)
            else:
                if not product.linked:
                    product.link(sort=sort)
        return self
    
    
    def link_all(self, sort='featured'):
        return self.link(sort=sort, force=True)
    
    
    def sync(self, force=False):
        
        changes = False
        for product in self.routine:
            if force:
                product.sync()
                changes = True
            else:
                if not product.synced:
                    product.sync()
                    changes = True
        if changes:
            self._analyze()
        else:
            return self
        
    def sync_all(self):
        return self.sync(force=True)
    
    def link_sync(self, sort='featured', force=False):
        self.link(sort=sort, force=force)
        self.sync(force=force)
        return self
    
    def link_sync_all(self, sort='featured'):
        return link_sync(self, sort=sort, force=True)
        
    def _analyze(self):
        """
        Tabulates frequency of ingredients across all Products
        """
        self._all_ingredients = []
        for product in self.routine:
            self._all_ingredients += [*product.ingredients]
        self._counts = Counter(self._all_ingredients)
        self.ingredients = list(set(self._all_ingredients))

        # make product vectors
        self._product_vectors = []
        for product in self.routine:
            product_vector = []
            for routine_ingredient in self.ingredients:
                if routine_ingredient in [*product.ingredients]:
                    product_vector.append(1)
                else:
                    product_vector.append(0)
            self._product_vectors.append(product_vector)
        return self
    
    def top_ingredients(self, top=None, mask=None):
        if mask is not None:
            masked_counts = Counter([x for x in self._all_ingredients if x in mask])
            return masked_counts.most_common(top)
        return self._counts.most_common(top)
    
    def has(self, ingredient):
        isolated_products = []
        for i, product_vector in enumerate(self._product_vectors):
            if product_vector[self.ingredients.index(ingredient)] == 1:
                isolated_products.append(self.routine[i].name)
        return isolated_products
        
    @property
    def cosdna_urls(self):
        return [product.cosdna_url for product in self.routine]
    
    @property
    def brands(self):
        return [product.brand for product in self.routine]
    
    @property
    def linked(self):
        try:
            return all([product.linked for product in self.routine])
        except:
            return False
    
    @property
    def synced(self):
        try:
            return all([product.synced for product in self.routine])
        except:
            return False
        
    def __str__(self):
        return f'Routine "{self.name}" with {len(self.routine)} products'
    
    def __repr__(self):
        return f'Routine(name={self.name}, routine={[product.name for product in self.routine]})'

In [ ]:
class RoutineDeep(CosDNA):
    
    def __init__(self, name=None, routine=None):
        super().__init__()
        self.name = name
        if routine is None:
            self.routine = []
        else:
            self.routine = []
            self.add(routine)
        
    def add(self, routine):
        routine = np.array([routine])
        routine = routine.ravel()
        for prod in routine:
            if type(prod) == Product:
                self.routine.append(prod)
            else:
                self.routine.append(Product(prod))
        
        return self
    
    def link(self, sort='featured', force=False):
        
        for product in self.routine:
            if force:
                product.link(sort=sort)
            else:
                if not product.linked:
                    product.link(sort=sort)
        return self
    
    
    def link_all(self, sort='featured'):
        return self.link(sort=sort, force=True)
    
    
    def sync(self, force=False, deep=False):
        
        changes = False
        for product in self.routine:
            if force:
                product.sync()
                changes = True
            else:
                if not product.synced:
                    product.sync()
                    changes = True
        if changes:
            self._analyze(deep=deep)
        else:
            return self
        
    def sync_all(self):
        return self.sync(force=True)
    
    def link_sync(self, sort='featured', force=False, deep=False):
        self.link(sort=sort, force=force)
        self.sync(force=force, deep=deep)
        return self
    
    def link_sync_all(self, sort='featured'):
        return link_sync(self, sort=sort, force=True)
        
    def _analyze(self, deep=False):
        """
        Tabulates frequency of ingredients across all Products
        """
        self.__all_ingredients = []
        self.__all_urls = []
        for product in self.routine:
            self.__all_ingredients += [*product.ingredients]
            self.__all_urls += [**product.ingredients.cosdna_url]
        self._counts = Counter(self._all_ingredients)
        self.ingredients = list(set(self._all_ingredients))

        # make product vectors
        self._product_vectors = []
        for product in self.routine:
            product_vector = []
            for routine_ingredient in self.ingredients:
                if routine_ingredient in [*product.ingredients]:
                    product_vector.append(1)
                else:
                    product_vector.append(0)
            self._product_vectors.append(product_vector)
        return self
    
    def top_ingredients(self, top=None, mask=None):
        if mask is not None:
            masked_counts = Counter([x for x in self._all_ingredients if x in mask])
            return masked_counts.most_common(top)
        return self._counts.most_common(top)
    
    def has(self, ingredient):
        isolated_products = []
        for i, product_vector in enumerate(self._product_vectors):
            if product_vector[self.ingredients.index(ingredient)] == 1:
                isolated_products.append(self.routine[i].name)
        return isolated_products
        
    @property
    def cosdna_urls(self):
        return [product.cosdna_url for product in self.routine]
    
    @property
    def brands(self):
        return [product.brand for product in self.routine]
    
    @property
    def linked(self):
        try:
            return all([product.linked for product in self.routine])
        except:
            return False
    
    @property
    def synced(self):
        try:
            return all([product.synced for product in self.routine])
        except:
            return False
        
    def __str__(self):
        return f'Routine "{self.name}" with {len(self.routine)} products'
    
    def __repr__(self):
        return f'Routine(name={self.name}, routine={[product.name for product in self.routine]})'

In [ ]:
routine_list = [
    'Biore Charcoal Acne Clearing Cleanser', 
    "kiehl's powerful strength line-reducing concentrate 12.5", 
    'Neutrogena Rapid Clear Spot Gel',
    'Roc Retinol Correxion eye cream', 
    'Body Merry Retinol Surge Moisturizer'
]

routine1 = Routine(routine=routine_list)
routine1.link_sync()
routine1._all_ingredients

In [ ]:
def routine_test(routine):
    
    tick = time.time()
    print(routine.routine)
    routine.link_sync()
    print()
    print(routine.brands)
    print(routine.has('water'))
    print(routine.top_ingredients(5))
    print(routine.add('la roche posay antihelios mineral'))
    routine.link_sync()
    print()
    print(routine.top_ingredients(5))
    tock = time.time()
    print()
    print(tock - tick)

In [ ]:
routine_test(routine1)

In [ ]:
test4 = Ingredient('titania')
test4.link_sync()
print(test4.name)

In [ ]:
test4.mass

In [ ]:
test4.cas_id

In [ ]:
test4.aliases

In [ ]:
test5 = Ingredient(cosdna_url='https://cosdna.com/eng/76298d372.html')
test5.link_sync()
test5.mass

In [ ]:
test6 = Ingredient('Sodium Pyrrolidone Carboxylic Acid')
test6.link_sync().name